## HW1: Word Embeddings

*based on [YSDA](https://github.com/yandexdataschool/nlp_course) materials*

Today we gonna play with word embeddings.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk gensim bokeh umap-learn```


In [1]:
import itertools
import string

import numpy as np
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [2]:
# download the data:
!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt -nc

Файл ‘./quora.txt’ уже существует; не загружается.


In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]
len(data)

537272

### Tokenization (1pt)

a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [167]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

# it is need in bonus task
lower_data = [text.lower() for text in data]
data_tok = [tokenizer.tokenize(text.lower()) for text in data]

Let's peek at the result:

In [6]:
' '.join(data_tok[0])

"can i get back with my ex even though she is pregnant with another guy ' s baby ?"

Small check that everything is alright

In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

### Word2Vec (1 pt)

 as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another NLP library that features many vector-based models incuding word2vec.

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=256,     # embedding vector size
                 min_count=5,     # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [9]:
# now you can get word vectors !
model.get_vector('anything')

array([-0.40272948,  0.1575108 , -0.49907094, -1.0233886 , -0.8773947 ,
       -0.21934721,  0.32948077,  0.977814  , -0.9902739 ,  0.13627139,
        0.51820654, -0.50633156, -0.91396564, -0.90979284, -0.5441526 ,
        0.5067336 , -0.48768687, -0.38792816,  0.8344079 , -1.166743  ,
       -1.2222135 , -0.4603727 , -0.45685035,  0.22496569, -1.3251657 ,
       -0.2504984 ,  0.13372758, -0.45569152, -0.12045481,  0.19244426,
        0.21002375,  0.44652152,  0.02595528,  1.0348179 ,  0.15424961,
        0.1860084 ,  0.20022063,  1.3847287 ,  0.41609186, -0.9847247 ,
        1.5453377 , -0.5670065 , -0.12237876, -0.35709125, -0.65868473,
        0.23302597, -1.4645231 , -0.9237661 , -0.51378983, -1.1284875 ,
        0.6772698 , -0.28636894, -0.68291587,  0.25998566,  1.8723716 ,
        0.3485582 , -0.8856377 ,  0.83317864,  0.90381825, -0.8976937 ,
        0.9224822 ,  0.08411061, -1.1037993 , -1.0586579 ,  0.06856426,
       -0.5728012 , -0.5688775 , -0.38658255,  0.30344015,  2.26

In [10]:
# or query similar words directly. Go play with it!
model.most_similar('blue')

[('red', 0.8602098226547241),
 ('yellow', 0.811117947101593),
 ('grey', 0.7807267904281616),
 ('gray', 0.7403100728988647),
 ('colored', 0.7119596600532532),
 ('color', 0.7107510566711426),
 ('white', 0.7092866897583008),
 ('brown', 0.7061247229576111),
 ('purple', 0.6953943371772766),
 ('colour', 0.6817023754119873)]

In [11]:
model.most_similar('love')

[('loved', 0.5709070563316345),
 ('asleep', 0.5639761090278625),
 ('friends', 0.559577465057373),
 ('happy', 0.5575960278511047),
 ('relationship', 0.5543469190597534),
 ('boyfriend', 0.54364413022995),
 ('feelings', 0.5406472682952881),
 ('flirting', 0.5265056490898132),
 ('girlfriend', 0.5264820456504822),
 ('loves', 0.5223814249038696)]

In [12]:
model.most_similar('mark')

[('zuckerberg', 0.7081819176673889),
 ('cuban', 0.5302456617355347),
 ('zuckerburg', 0.5240932106971741),
 ('larry', 0.5219124555587769),
 ('marked', 0.5187152624130249),
 ('williams', 0.5127007961273193),
 ('jimmy', 0.4964846670627594),
 ('wikipedia', 0.49544981122016907),
 ('howard', 0.4806172847747803),
 ('ama', 0.47917506098747253)]

Show the linear structure in embeddings space with 2-3 examples

In [13]:
model.most_similar(positive="tea", negative="coffee")

[('entered', 0.357553094625473),
 ('slots', 0.33785364031791687),
 ('severe', 0.33736881613731384),
 ('swings', 0.3361569344997406),
 ('card', 0.3314844071865082),
 ('excessive', 0.32733970880508423),
 ('hazel', 0.3263992667198181),
 ('berets', 0.3246190547943115),
 ('pus', 0.3230167031288147),
 ('crooked', 0.3187481760978699)]

In [14]:
model.most_similar(positive="hate", negative="happy")

[('deny', 0.50898677110672),
 ('dislike', 0.4696238934993744),
 ('oppose', 0.4264301359653473),
 ('support', 0.42491403222084045),
 ('among', 0.41166040301322937),
 ('admire', 0.4042936861515045),
 ('expose', 0.3992456793785095),
 ('worship', 0.39382001757621765),
 ('enlighten', 0.3920295834541321),
 ('tolerate', 0.39024463295936584)]

In [15]:
model.most_similar(positive="mommy", negative="father")

[('kpis', 0.7098230123519897),
 ('kangaroos', 0.6996870636940002),
 ('comsol', 0.6970863342285156),
 ('invertebrates', 0.6954016089439392),
 ('2k17', 0.693355143070221),
 ('notebooks', 0.6859063506126404),
 ('designations', 0.6854773163795471),
 ('extrusive', 0.6851963400840759),
 ('nrg', 0.6849888563156128),
 ('pluralsight', 0.6843746900558472)]

### Glove + PCA (2 pt)

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [16]:
import gensim.downloader as api

model = api.load("glove-twitter-25") # download glove embeddings with 25 dimension trained on twitter 

In [17]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('realtor', 0.8265186548233032),
 ('gfx', 0.8249695897102356),
 ('caterers', 0.798202395439148),
 ('beatmaker', 0.7936854362487793),
 ('recruiter', 0.7892400026321411),
 ('sfi', 0.784467339515686),
 ('sosh', 0.7840632796287537),
 ('promoter', 0.7838250994682312),
 ('smallbusiness', 0.7786215543746948),
 ('promoters', 0.77646803855896)]

In [18]:
model.most_similar(positive=["coder", "monkey"], negative=["sql"])

[('doggy', 0.8787190914154053),
 ('husky', 0.8757146000862122),
 ('squirrel', 0.8662303686141968),
 ('puppy', 0.8291974067687988),
 ('pug', 0.8216956853866577),
 ('koala', 0.8212488293647766),
 ('cock', 0.8182564377784729),
 ('corgi', 0.8171593546867371),
 ('kitten', 0.8148956298828125),
 ('redhead', 0.8120709657669067)]

In [19]:
model.most_similar(positive=["promoter", "brain"], negative=["keys"])

[('bilingual', 0.8574241995811462),
 ('psychologist', 0.8413557410240173),
 ('counseling', 0.8243622779846191),
 ('part-time', 0.8222078084945679),
 ('advocate', 0.8208866715431213),
 ('caregiver', 0.8187721371650696),
 ('counselor', 0.8180349469184875),
 ('clinical', 0.8160943984985352),
 ('physicians', 0.8160588145256042),
 ('practitioners', 0.8126274943351746)]

In [20]:
model.most_similar(positive=["recruiter", "brain"], negative=["stupid"])

[('merchandiser', 0.9047132730484009),
 ('automation', 0.8962619304656982),
 ('specialist', 0.8875490427017212),
 ('technician', 0.8842819333076477),
 ('compliance', 0.8811959028244019),
 ('procurement', 0.8809123039245605),
 ('rehabilitation', 0.8800356984138489),
 ('configuration', 0.8768974542617798),
 ('generalist', 0.8767427206039429),
 ('implementation', 0.8756457567214966)]

In [21]:
model.most_similar(positive=["elon", "musk", "rocket"])

[('herschel', 0.8975597620010376),
 ('butcher', 0.8948329091072083),
 ('tesla', 0.8921502232551575),
 ('edison', 0.8739283680915833),
 ('rza', 0.8662421703338623),
 ('spacex', 0.8624916672706604),
 ('clancy', 0.8615334630012512),
 ('baron', 0.8581820726394653),
 ('deere', 0.8565327525138855),
 ('alibaba', 0.8542810678482056)]

In [22]:
words = sorted(model.key_to_index.keys(), 
               key=lambda word: model.get_vecattr(word, "count"),
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [23]:
# for each word, compute it's vector with model

word_vectors = np.array([model.get_vector(word) for word in words])

In [24]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 25)
assert np.isfinite(word_vectors).all()

In [25]:
word_vectors.shape

(1000, 25)

Use PCA to decrease embeddings dimension so that we will be able to draw them.

In [26]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)
scaler = StandardScaler()
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca.fit(word_vectors)
down_demention_vectors = pca.transform(word_vectors)
scaler.fit(down_demention_vectors)
word_vectors_pca = scaler.transform(down_demention_vectors)

In [27]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

Function to draw embeddings

In [28]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [29]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1005', ...)

### Phrase Embeddings (2 pt)

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.


In [55]:
def get_phrase_embedding(model, phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    tokenized_phrase = tokenizer.tokenize(phrase.lower())
    word_vectors = []
    for word in tokenized_phrase:
        if word in model.key_to_index:
            word_vectors.append(model.get_vector(word))
    
    if len(word_vectors) == 0:
        return 0;

    vector = np.average(word_vectors, axis=0)
    
    return vector
        
    

In [58]:
print(data[402687])
get_phrase_embedding(model, data[402687])

What gift should I give to my girlfriend on her birthday?



array([-0.18204999,  0.30953574,  0.20861094,  0.07982156, -0.22565515,
       -0.33001748,  1.2495784 ,  0.13134292, -0.33788875,  0.06196944,
       -0.231793  ,  0.09389219, -4.9685497 , -0.23611419, -0.32609668,
       -0.092073  ,  0.4407505 , -0.75413746, -0.5389092 , -0.184752  ,
        0.07867809,  0.20018655, -0.16202375,  0.30375698, -0.41255665],
      dtype=float32)

In [59]:
vector = get_phrase_embedding(model, "I'm very sure. This never happened to me before...")

In [60]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases and turn them to numpy array
phrase_vectors = np.asarray([get_phrase_embedding(model, x) for x in chosen_phrases])

In [61]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [62]:
phrase_vectors.shape

(1001, 25)

In [63]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = scaler.transform(pca.transform(phrase_vectors))

In [64]:
phrase_vectors_2d.shape, np.linalg.norm(phrase_vectors_2d[-1])

((1001, 2), 0.8437346)

In [65]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1319', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [66]:
# filter all sentences that include at least one NONPRINTABLE char
# compute vector embedding for all remaining lines in data

printable_set = set(string.printable)

data_subset = []

for phrase in data:
    if set(phrase).issubset(printable_set):
        data_subset.append(phrase)


In [67]:
trstr = "How do I get to the dark web?\n"
len(data), len(data_subset), trstr in data_subset

(537272, 531050, True)

In [68]:
get_phrase_embedding(model, trstr).shape[0]

25

In [145]:
data_vectors = []
origin_phrases = []
for phrase in data_subset:
    res = get_phrase_embedding(model, phrase)
    if isinstance(res, int):
        print("woops")
        continue
    origin_phrases.append(phrase)
    data_vectors.append(res)
    
data_vectors = np.array(data_vectors, dtype=np.ndarray)
origin_phrases = np.array(origin_phrases, dtype=str)

# set_vectors = set(data_vectors) # set of all embedding vectors

norms = np.array([np.linalg.norm(vec) for vec in data_vectors])# np.array of all norms for data_vectors

print(data_vectors.shape)

woops
woops
(531048, 25)


In [146]:
# Let's check some sh@t (sorry, I had to spend too much time on this)
assert(data_vectors.shape[1] == 25)
assert(get_phrase_embedding(model, trstr) in data_vectors)

In [147]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vector = get_phrase_embedding(model, query)
    if query_vector.size == 0:
        return
    
    query_norm = np.linalg.norm(query_vector)
    cos_sim = (data_vectors @ query_vector) / (query_norm * norms)
    indexes = np.argsort(cos_sim)[::-1][:k]
    
    print(max(cos_sim))
    return (cos_sim[indexes], np.array(origin_phrases)[indexes])# <YOUR CODE: top-k lines starting from most similar>

In [148]:
trstrvec = get_phrase_embedding(model, trstr)
trstrnorm = np.linalg.norm(trstrvec)

ind = 0 #344676

assert(trstrvec in data_vectors)

for i in range(len(data_vectors)):
    if (data_vectors[i] == trstrvec).all():
        ind = i
        print("Find! : ", ind)
        
data_vectors[ind] @ trstrvec / (norms[ind]*trstrnorm) , find_nearest(trstr, 1)[0]# What?

Find! :  344676
0.9999999893088634


(0.9999999893088634, array([0.9999999893088634], dtype=object))

In [149]:
cos_sims, results = find_nearest(query="How do i enter the matrix?", k=10)

for cos, phrase in zip(cos_sims, results):
    print("cos_angle: ", cos, "\nphrase: ", phrase)

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'

0.99777370912379
cos_angle:  0.99777370912379 
phrase:  How do I get to the dark web?

cos_angle:  0.9964147876430194 
phrase:  What universal remote do I need and how do I set it up to a Blaupunkt TV?

cos_angle:  0.9963033574253181 
phrase:  How do I connect the ASUS_T00Q to my PC?

cos_angle:  0.9959279928171391 
phrase:  How do you print the gridlines in Excel 2007?

cos_angle:  0.9959279928171391 
phrase:  How do you print the gridlines in Excel 2003?

cos_angle:  0.9959279928171391 
phrase:  How do you print the gridlines in Excel 2010?

cos_angle:  0.9958131512732857 
phrase:  I would like to create a new website. What do I have to do?

cos_angle:  0.9957309611336619 
phrase:  How do I get the new Neko Atsume wallpapers? How do they work?

cos_angle:  0.9957022396472184 
phrase:  I want to experience the 4G network. Do I need to change my SIM card from 3G to 4G?

cos_angle:  0.9953885374667286 
phrase:  What do I have to do to sell my photography?



In [150]:
find_nearest(query="How does Trump?", k=10)[1]

0.9909127417475514


array(['What does Donald Trump think about Israel?\n',
       'Who or what is Donald Trump, really?\n',
       'Donald Trump: Why are there are so many questions about Donald Trump on Quora?\n',
       'Does anyone like Trump and Clinton?\n',
       'What does Cortana mean?\n',
       'Did Bill Gates outcompete and outsmart IBM? Why? How?\n',
       'Why and how is Bill Gates so rich?\n',
       'What does Donald Trump think about Pakistan?\n',
       'What do you think about Trump and Obama?\n',
       'Who and what is Quora?\n'], dtype='<U1170')

In [151]:
find_nearest(query="Why don't i ask a question myself?", k=10)[1]

0.9949912044266839


array(["Why don't my parents listen to me?\n",
       "Why don't people appreciate me?\n",
       "Why she don't interact with me?\n", "Why don't I get a date?\n",
       "Why don't I get a girlfriend?\n",
       "Why don't I have a girlfriend?\n",
       "Why don't I have a boyfriend?\n",
       "Why don't I like people touching me?\n",
       "Why can't I ask a question anonymously?\n",
       "Why don't you use Facebook much?\n"], dtype='<U1170')

### Phrase Embeddings with TfIdf (3 pt, bonus)

Improve the above __get_phrase_embedding__ function to use TF-IDF as vector weights.

1) train sklearn tf-idf implementation
2) update function *get_phrase_embedding*
3) compare new and old versions using *find_nearest* on 2-3 examples 

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer(max_features=10000)
TFIDF = vectorizer.fit_transform(lower_data)

In [214]:
def get_weight(word):
    return df[df.index == word]['TF-IDF'].values

TFIDF[1].

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

### FastText (3 pt, bonus)

Implement search enginge using [FastText](https://fasttext.cc/docs/en/python-module.html) embeddings. 

1) train fasttext embeddings on all data
2) update *get_phrase_embeddings* function
3) implement *find_nearest* function with fasttext embeddings
4) compare results with glove version from the main task

In [ ]:
# YOUR CODE HERE